<a href="https://colab.research.google.com/github/kiyong21c/pytorch_tutorial/blob/main/20220707_Softmax_CrossEntrophy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# softmax, crossentropy

import torch
import torch.nn as nn
import numpy as np

#
#        -> 2.0              -> 0.65  
# Linear -> 1.0  -> Softmax  -> 0.25   -> CrossEntropy(y, y_hat)
#        -> 0.1              -> 0.1                   
#
#     scores(logits)      probabilities
#                           sum = 1.0
#

# Softmax applies the exponential function to each element, and normalizes
# by dividing by the sum of all these exponentials
# -> squashes the output to be between 0 and 1 = probability(확률)
# sum of all probabilities is 1
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.array([2.0, 1.0, 0.1]) # 1차원
outputs = softmax(x)
print('softmax numpy:', outputs)

x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0) # dim=0 : 행끼리의 합이 1이 되도록
print('softmax torch:', outputs)

x = torch.tensor([[2.0], [1.0], [0.1]]) # 2차원 : (3, 1)
outputs = torch.softmax(x, dim=0) # dim=0 : 행끼리의 합이 1이 되도록
print('softmax torch:', outputs)

# Cross entropy
# Cross-entropy loss, or log loss, measures the performance of a classification model 
# whose output is a probability value between 0 and 1. 
# -> loss increases as the predicted probability diverges from the actual label
def cross_entropy(actual, predicted):   # 손실함수 : 손실값을 반환
    EPS = 1e-15
    predicted = np.clip(predicted, EPS, 1 - EPS)
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0])

# y must be one hot encoded
# if class 0: [1 0 0]
# if class 1: [0 1 0]
# if class 2: [0 0 1]
Y = np.array([1, 0, 0])                     # 실제타깃값 : [1,   0,   0]
Y_pred_good = np.array([0.7, 0.2, 0.1])     # 좋은예측값 : [0.7, 0.2, 0.1]
Y_pred_bad = np.array([0.1, 0.3, 0.6])      # 나쁜예측값 : [0.1, 0.3, 0.6]
l1 = cross_entropy(Y, Y_pred_good)      # 좋은 예측값일때 loss
l2 = cross_entropy(Y, Y_pred_bad)       # 나쁜 예측값일때 loss
print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')

# CrossEntropyLoss in PyTorch (applies Softmax)
# nn.CrossEntropyLoss = nn.LogSoftmax(=log+softmax) + nn.NLLLoss(=negative)
loss = nn.CrossEntropyLoss()
# loss(input, target)

# target is of size nSamples = 1
# each element has class label: 0, 1, or 2
# Y (=target) contains class labels, not one-hot
Y = torch.tensor([0]) # 실제타깃값이 one-hot 인코딩된 값이 아님, 예측값의 인덱스와 매칭시킬예정

# input is of size nSamples x nClasses = 1 x 3
# y_pred (=input) must be raw, unnormalizes scores (logits) for each class, not softmax
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]]) # 예측값은 "행", 소프트맥스 적용X : CrossEntropyLoss()의 내부 로직을 통해 softmax가 적용될 예정
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]]) # 예측값은 "행", 소프트맥스 적용X : CrossEntropyLoss()의 내부 로직을 통해 softmax가 적용될 예정
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'PyTorch Loss1: {l1.item():.4f}')
print(f'PyTorch Loss2: {l2.item():.4f}')


# get predictions
# print(torch.max(Y_pred_good, dim=1)) # torch.return_types.max(values=tensor([2.]), indices=tensor([0])) : 가장 큰 값, 해당 인덱스
# dim=0 : 행을기준으로, dim=1 : 열을기준으로 가장 큰 값
_, predictions1 = torch.max(Y_pred_good, 1) # 가장 큰값의 인덱스만
_, predictions2 = torch.max(Y_pred_bad, 1)  # 가장 큰값의 인덱스만
print(f'Actual class: {Y.item()}, Y_pred1: {predictions1.item()}, Y_pred2: {predictions2.item()}')

# allows batch loss for multiple samples

# target is of size nBatch = 3(3Samples)
# each element has class label: 0, 1, or 2
Y = torch.tensor([2, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# Y_pred are logits (not softmax)
Y_pred_good = torch.tensor(
    [[0.1, 0.2, 3.9], # predict class 2
    [1.2, 0.1, 0.3], # predict class 0
    [0.3, 2.2, 0.2]]) # predict class 1

Y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
    [0.1, 0.3, 1.5],
    [1.2, 0.2, 0.5]])

# get loss
l1 = loss(Y_pred_good, Y) # loss = nn.CrossEntropyLoss()
l2 = loss(Y_pred_bad, Y)
print(f'Batch Loss1:  {l1.item():.4f}')
print(f'Batch Loss2: {l2.item():.4f}')

# get predictions class(label/target)
_, predictions1 = torch.max(Y_pred_good, 1) # predictions1.shape : torch.size([3])
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y}, Y_pred1: {predictions1}, Y_pred2: {predictions2}')

# Binary classification
class NeuralNet1(nn.Module): # 사용자정의 모델생성
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) # 입력층
        self.relu = nn.ReLU() # 활성화함수층
        self.linear2 = nn.Linear(hidden_size, 1)  # 출력층
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # sigmoid at the end
        y_pred = torch.sigmoid(out) # 이진분류용 모델이므로 sigmoid 적용
        return y_pred

model = NeuralNet1(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss() # 이진분류용 손실함수(softmax적용 안함, 따라서 forward에서 sigmoid적용)

# # Multiclass problem
# class NeuralNet2(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(NeuralNet2, self).__init__()
#         self.linear1 = nn.Linear(input_size, hidden_size) 
#         self.relu = nn.ReLU()
#         self.linear2 = nn.Linear(hidden_size, num_classes)  
    
#     def forward(self, x):
#         out = self.linear1(x)
#         out = self.relu(out)
#         out = self.linear2(out)
#         # no softmax at the end
#         return out

# # model = NeuralNet2(input_size=28*28, hidden_size=5, num_classes=3)
# # criterion = nn.CrossEntropyLoss()  # model을통한 output 즉, 예측값에 softmax가 적용되는 로직을 갖는 손실함수인 CrossEntropy()를 사용하므로, 모델 정의할 때 마지막에 softmax를 적용하지 않음
# # criterion(model(X), Y) : model(X)는 output 즉, 예측값이며, CrossEntrophy 로직을 통해 softmax가 적용됨

softmax numpy: [0.65900114 0.24243297 0.09856589]
softmax torch: tensor([0.6590, 0.2424, 0.0986])
softmax torch: tensor([[0.6590],
        [0.2424],
        [0.0986]])
Loss1 numpy: 0.3567
Loss2 numpy: 2.3026
PyTorch Loss1: 0.4170
PyTorch Loss2: 1.8406
Actual class: 0, Y_pred1: 0, Y_pred2: 1
Batch Loss1:  0.2834
Batch Loss2: 1.6418
Actual class: tensor([2, 0, 1]), Y_pred1: tensor([2, 0, 1]), Y_pred2: tensor([0, 2, 0])
torch.Size([3])
